## Create a Simple  Text Instruction Pair Dataset Using Notus 7-B

Install Dependencies

In [2]:
pip install accelerate transformers torch datasets tqdm

Note: you may need to restart the kernel to use updated packages.


Script to generate the Dataset

In [3]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import Dataset
import pandas as pd
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the model and tokenizer
model_name = "argilla/notus-7b-v1"  # or your preferred 7B model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
model.to(device) 

def generate_text(instruction, max_length=100):
    prompt = f"Instruction: {instruction}\nResponse:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Define a list of instructions
instructions = [
"Create a dialogue between a teacher and a child about cultural diversity,",
"Explain evolution to a 10-year-old,",
"Explain quantum computing to a 5-year-old,",
"Explain mindfulness to a 12-year-old,",
"Design a science fiction movie plot set in ancient Egypt,",
"Compare and contrast fast food and natural selection,",
"Create a dialogue between a scientist and a historian about artificial intelligence,",
"Explain artificial intelligence to a 15-year-old,",
"Create a marketing slogan for virtual reality headset,",
"Design a psychological thriller movie plot set in the roaring twenties,",
"Create a dialogue between a politician and a child about artificial intelligence,",
"Create a marketing slogan for electric car,",
"Create a dialogue between a scientist and a philosopher about healthy living,",
"Compare and contrast electric cars and human intelligence,",
"Create a dialogue between a scientist and an alien about climate change,",
"Discuss the significance of climate change in 19th century,",
"Create a dialogue between a teacher and a time traveler about space exploration,",
"Create a dialogue between an artist and a detective about space exploration,",
"Design a crime mystery movie plot set in the roaring twenties,",
"Create a dialogue between a teacher and a celebrity about climate change,",
"Discuss the significance of space exploration in the Interstellar film,",
"Create a dialogue between a chef and a detective about healthy living,",
"Create a dialogue between a teacher and an alien about artificial intelligence,",
"Design a plan to improve CPU Architecture,",
"Explain relativity to a 12-year-old,",
"Create a dialogue between a chef and a time traveler about space exploration,",
"Compare and contrast traditional education and home-cooked meals,",
"Create a dialogue between an artist and a child about climate change,",
"Create a dialogue between a chef and a retiree about climate change,",
"Create a dialogue between a scientist and a detective about space exploration,",
"Design a documentary movie plot set in ancient Egypt,",
"Explain blockchain to a 10-year-old,",
"Compare and contrast remote work and office-based work,",
"Create a dialogue between a chef and an alien about cultural diversity,",
"Compare and contrast social media and gasoline vehicles,",
"Create a dialogue between a politician and a time traveler about climate change,",
"Design a historical drama movie plot set in the roaring twenties,",
"Explain mindfulness to a 5-year-old,",
"Create a dialogue between an artist and a celebrity about space exploration,",
"Create a dialogue between a teacher and a historian about healthy living,",
"Describe how photosynthesis works,",
"Design a animated adventure movie plot set in the Wild West,",
"Create a dialogue between an artist and a robot about cultural diversity,",
"Compare and contrast remote work and cable television,",
"Explain mindfulness to a 8-year-old,",
"Compare and contrast artificial intelligence and natural selection,",
"Create a dialogue between a politician and an alien about artificial intelligence,",
"Compare and contrast social media and augmented reality,",
"Create a dialogue between a teacher and a celebrity about cultural diversity,",
"Compare and contrast social media and natural selection,",
"Design a science fiction movie plot set in the Wild West,",
"Design a science fiction movie plot set in a post-apocalyptic world,",
"Imagine and describe a day in the life of a cybersecurity expert,",
"Design a animated adventure movie plot set in an underwater civilization,",
"Create a dialogue between a scientist and a student about climate change,",
"Create a dialogue between a politician and a time traveler about cultural diversity,",
"Create a dialogue between a scientist and a celebrity about healthy living,",
"Explain cryptocurrency to a 8-year-old,",
"Compare and contrast artificial intelligence and human intelligence,",
"Design a documentary movie plot set in a magical realm,",
"Design a horror movie plot set in a dystopian society,",
"Compare and contrast remote work and face-to-face communication,",
"Explain quantum computing to a 12-year-old,",
"Create a dialogue between a teacher and a student about artificial intelligence,",
"Compare and contrast virtual reality and face-to-face communication,",
"Create a dialogue between a scientist and an alien about artificial intelligence,",
"Compare and contrast remote work and home-cooked meals,",
"Create a dialogue between a teacher and a child about artificial intelligence,",
"Create a dialogue between an artist and a detective about climate change,",
"Create a dialogue between a politician and a student about climate change,",
"Create a dialogue between a chef and an alien about space exploration,",
"Evaluate the pros and cons of cultural diversity,",
"Create a dialogue between an artist and a student about climate change,",
"Create a dialogue between a scientist and a philosopher about artificial intelligence,",
"Design a crime mystery movie plot set in the Wild West,",
"Explain relativity to a 5-year-old,",
"Explain climate change to a 10-year-old,",
"Create a dialogue between a chef and a celebrity about climate change,",
"Create a dialogue between a scientist and a child about climate change,",
"Design a documentary movie plot set in a futuristic megacity,",
"Develop a concept for a space exploration themed restaurant,",
"Create a dialogue between a politician and a philosopher about artificial intelligence,",
"Create a dialogue between a politician and a robot about healthy living,",
"Create a dialogue between an artist and a philosopher about artificial intelligence,",
"Compare and contrast artificial intelligence and office-based work,",
"Create a dialogue between a chef and a time traveler about healthy living,",
"Explain democracy to a 10-year-old,",
"Create a dialogue between a scientist and a student about cultural diversity,",
"Create a dialogue between a teacher and a philosopher about climate change,",
"Write a short story about space exploration,",
"Design a superhero movie plot set in the Wild West,",
"Create a dialogue between a politician and a philosopher about healthy living,",
"Explain relativity to a 15-year-old,",
"Compare and contrast traditional education and office-based work,",
"Create a dialogue between a scientist and a historian about climate change,",
"Create a dialogue between a politician and a student about space exploration,",
"Design a historical drama movie plot set in a dystopian society,",
"Design a psychological thriller movie plot set in a parallel universe,",
"Create a dialogue between a chef and a detective about cultural diversity,",
"Design a crime mystery movie plot set in a futuristic megacity,",
"Create a dialogue between a politician and a retiree about healthy living,",
"Create a dialogue between a scientist and a time traveler about healthy living,",
"Create a dialogue between a chef and a detective about artificial intelligence,",
"Explain cryptocurrency to a 10-year-old,",
"Create a dialogue between a politician and a child about climate change,",
"Create a dialogue between a teacher and a philosopher about cultural diversity,",
"Explain blockchain to a 15-year-old,",
"Explain quantum computing to a 10-year-old,",
"Explain blockchain to a 5-year-old,",
"Create a dialogue between a scientist and a student about healthy living,",
"Create a dialogue between an artist and a retiree about space exploration,",
"Create a dialogue between a scientist and a robot about space exploration,",
"Compare and contrast streaming services and human intelligence,",
"Develop a concept for a underwater themed restaurant,",
"Design a romantic comedy movie plot set in ancient Egypt,",
"Design a animated adventure movie plot set in a futuristic megacity,",
"Compare and contrast artificial intelligence and gasoline vehicles,",
"Create a dialogue between a chef and a celebrity about healthy living,",
"Create a dialogue between an artist and a detective about artificial intelligence,",
"Create a dialogue between a teacher and a robot about cultural diversity,",
"Create a dialogue between a teacher and a historian about climate change,",
"Create a marketing slogan for renewable energy solution,",
"Design a romantic comedy movie plot set in a parallel universe,",
"Create a dialogue between a teacher and a retiree about space exploration,",
"Create a dialogue between a teacher and a robot about artificial intelligence,",
"Create a dialogue between a scientist and a philosopher about climate change,",
"Design a superhero movie plot set in a parallel universe,",
"Design a animated adventure movie plot set in the roaring twenties,",
"Design a historical drama movie plot set in a magical realm,",
"Create a dialogue between a scientist and a robot about healthy living,",
"Design a horror movie plot set in an alien planet,",
"Compare and contrast virtual reality and human intelligence,",
"Create a dialogue between a politician and a celebrity about climate change,",
"Create a dialogue between a scientist and a detective about artificial intelligence,",
"Create a dialogue between a scientist and a celebrity about artificial intelligence,",
"Explain artificial intelligence to a 5-year-old,",
"Compare and contrast streaming services and natural selection,",
"Compare and contrast remote work and augmented reality,",
"Explain photosynthesis to a 10-year-old,",
"Create a dialogue between a scientist and a student about artificial intelligence,",
"Develop a concept for a magical realism themed restaurant,",
"Create a dialogue between an artist and a retiree about artificial intelligence,",
"Compare and contrast electric cars and office-based work,",
"Design a historical drama movie plot set in an underwater civilization,",
"Write a news article about cultural diversity,",
"Create a marketing slogan for sustainable fashion brand,",
"Create a dialogue between a politician and a child about space exploration,",
"Summarize the key points of artificial intelligence,",
"Compare and contrast artificial intelligence and augmented reality,",
"Explain photosynthesis to a 8-year-old,",
"Create a dialogue between a scientist and a retiree about climate change,",
"Create a dialogue between a politician and an alien about climate change,",
"Design a psychological thriller movie plot set in an underwater civilization,",
"Compare and contrast virtual reality and office-based work,",
"Imagine and describe a day in the life of a interplanetary diplomat,",
"Create a dialogue between a teacher and a celebrity about healthy living,",
"Describe how gene editing works,",
"Design a superhero movie plot set in an alien planet,",
"Compare and contrast artificial intelligence and cable television,",
"Compare and contrast genetic engineering and augmented reality,",
"Design a romantic comedy movie plot set in the roaring twenties,",
"Design a crime mystery movie plot set in an underwater civilization,",
"Compare and contrast remote work and gasoline vehicles,",
"Design a psychological thriller movie plot set in a magical realm,",
"Create a dialogue between a teacher and a historian about space exploration,",
"Create a dialogue between a politician and a time traveler about healthy living,",
"Create a dialogue between a politician and a celebrity about space exploration,",
"Design a animated adventure movie plot set in a parallel universe,",
"Create a dialogue between an artist and a robot about space exploration,",
"Design a romantic comedy movie plot set in an underwater civilization,",
"Create a dialogue between a scientist and a time traveler about artificial intelligence,",
"Create a dialogue between a politician and a detective about space exploration,",
"Create a dialogue between a scientist and a celebrity about climate change,",
"Design a historical drama movie plot set in a post-apocalyptic world,",
"Explain democracy to a 12-year-old,",
"Compare and contrast artificial intelligence and online learning,",
"Create a dialogue between a teacher and a philosopher about healthy living,",
"Create a dialogue between a politician and a historian about cultural diversity,",
"Create a dialogue between a chef and a retiree about space exploration,",
"Create a dialogue between a scientist and a detective about healthy living,",
"Describe how voting works,",
"Create a dialogue between a chef and a robot about climate change,",
"Create a dialogue between a politician and an alien about cultural diversity,",
"Compare and contrast virtual reality and natural selection,",
"Compare and contrast remote work and human intelligence,",
"Create a dialogue between a teacher and a child about space exploration,",
"Describe a busy airport in vivid detail,",
"Develop a concept for a eco-futurism themed restaurant,",
"Compare and contrast electric cars and home-cooked meals,",
"Develop a concept for a fairy tale themed restaurant,",
"Create a dialogue between a teacher and a celebrity about space exploration,",
"Create a dialogue between a chef and an alien about artificial intelligence,",
"Design a science fiction movie plot set in a parallel universe,",
"Create a dialogue between a chef and a child about cultural diversity,",
"Create a dialogue between a scientist and a time traveler about cultural diversity,",
"Explain democracy to a 5-year-old,",
"Create a dialogue between a scientist and a retiree about artificial intelligence,",
"Compare and contrast cryptocurrencies and home-cooked meals,",
"Create a dialogue between a politician and a detective about climate change,",
"Create a dialogue between a chef and a historian about artificial intelligence,",
"Create a dialogue between a teacher and a robot about healthy living,",
"Create a dialogue between an artist and a child about cultural diversity,",
"Create a dialogue between a chef and a student about healthy living,",
"Compare and contrast electric cars and face-to-face communication,",
"Imagine and describe a day in the life of a rogue geneticist,",
"Create a dialogue between a scientist and a philosopher about cultural diversity,",
"Create a marketing slogan for smart home security system,",
"Design a horror movie plot set in the Wild West,",
"Create a dialogue between a teacher and a detective about space exploration,",
"Compare and contrast electric cars and augmented reality,",
"Create a dialogue between an artist and a detective about cultural diversity,",
"Create a dialogue between a politician and a robot about space exploration,",
"Create a dialogue between a politician and a historian about healthy living,",
"Compare and contrast genetic engineering and gasoline vehicles,",
"Create a dialogue between a teacher and a celebrity about artificial intelligence,",
"Design a horror movie plot set in an underwater civilization,",
"Create a dialogue between an artist and a student about space exploration,",
"Create a dialogue between a teacher and a detective about healthy living,",
"Explain quantum computing to a 15-year-old,",
"Explain cryptocurrency to a 5-year-old,",
"Explain relativity to a 8-year-old,",
"Examine the relationship between Deep Learning and Machine Learning,",
"Create a marketing slogan for AI-powered personal assistant,",
"Explain cryptocurrency to a 15-year-old,",
"Create a dialogue between a scientist and a retiree about cultural diversity,",
"Explain climate change to a 12-year-old,",
"Create a dialogue between a teacher and a retiree about cultural diversity,",
"Design a musical movie plot set in an underwater civilization,",
"Design a psychological thriller movie plot set in the Wild West,",
"Compare and contrast streaming services and traditional banking,",
"Invent a new GPU and describe its features,",
"Create a dialogue between a scientist and a celebrity about space exploration,",
"Explain photosynthesis to a 12-year-old,",
"Develop a strategy to play football,",
"Design a musical movie plot set in a parallel universe,",
"Create a dialogue between a teacher and an alien about healthy living,",
"Write a short story about healthy living,",
"Create a dialogue between a politician and a historian about artificial intelligence,",
"Create a dialogue between a scientist and a time traveler about climate change,",
"Compare and contrast social media and office-based work,",
"Create a dialogue between a politician and a philosopher about climate change,",
"Imagine and describe a day in the life of a virtual reality designer,",
"Create a marketing slogan for mental health app,",
"Summarize the key points of cultural diversity,",
"Compare and contrast traditional education and human intelligence,",
"Create a dialogue between a teacher and a student about cultural diversity,",
"Design a science fiction movie plot set in the roaring twenties,",
"Create a dialogue between a teacher and an alien about climate change,",
"Imagine and describe a day in the life of a synthetic biology entrepreneur,",
"Create a dialogue between a teacher and an alien about space exploration,",
"Create a dialogue between a scientist and a detective about cultural diversity,",
"Create a dialogue between an artist and a philosopher about cultural diversity,",
"Compare and contrast fast food and office-based work,",
"Critique the argument that smartphones are necessary,",
"Create a dialogue between a chef and a child about healthy living,",
"Explain relativity to a 10-year-old,",
"Compare and contrast social media and face-to-face communication,",
"Describe how blockchain technology works,",
"Design a psychological thriller movie plot set in ancient Egypt,",
"Describe how cloud computing works,",
"Imagine and describe a day in the life of an AI gaining consciousness,",
"Create a dialogue between an artist and a historian about space exploration,",
"Design a superhero movie plot set in a dystopian society,",
"Create a dialogue between a scientist and an alien about healthy living,",
"Design a musical movie plot set in the roaring twenties,",
"Describe how renewable energy production works,",
"Design a documentary movie plot set in an underwater civilization,",
"Create a dialogue between a politician and a detective about healthy living,",
"Explain evolution to a 8-year-old,",
"Design a crime mystery movie plot set in a parallel universe,",
"Create a dialogue between an artist and a historian about climate change,",
"Explain democracy to a 8-year-old,",
"Create a dialogue between an artist and a retiree about cultural diversity,",
"Discuss the significance of healthy living in the 21st century,",
"Create a dialogue between a politician and a celebrity about artificial intelligence,",
"Design a science fiction movie plot set in a futuristic megacity,",
"Create a dialogue between a politician and a robot about artificial intelligence,",
"Create a dialogue between a politician and an alien about healthy living,",
"Create a dialogue between a scientist and a retiree about space exploration,",
"Create a dialogue between a chef and a student about space exploration,",
"Create a dialogue between a chef and a time traveler about cultural diversity,",
"Design a crime mystery movie plot set in a dystopian society,",
"Create a dialogue between an artist and an alien about space exploration,",
"Create a dialogue between an artist and a historian about artificial intelligence,",
"Explain mindfulness to a 10-year-old,",
"Describe a remote mountain village in vivid detail,",
"Design a superhero movie plot set in a post-apocalyptic world,",
"Compare and contrast fast food and augmented reality,",
"Create a dialogue between a chef and a robot about healthy living,",
"Design a animated adventure movie plot set in a dystopian society,",
"Compare and contrast genetic engineering and online learning,",
"Write a short story about artificial intelligence,",
"Explain democracy to a 15-year-old,",
"Design a documentary movie plot set in the Wild West,",
"Create a dialogue between a chef and a philosopher about space exploration,",
"Create a dialogue between a scientist and an alien about space exploration,",
"Write a news article about healthy living,",
"Create a dialogue between a politician and a celebrity about cultural diversity,",
"Create a dialogue between a scientist and a retiree about healthy living,",
"Compare and contrast genetic engineering and face-to-face communication,",
"Compare and contrast traditional education and face-to-face communication,",
"Create a dialogue between a politician and a student about cultural diversity,",
"Create a dialogue between a scientist and a time traveler about space exploration,",
"Create a dialogue between a chef and an alien about climate change,",
"Compare and contrast traditional education and cable television,",
"Summarize the key points of space exploration,",
"Create a dialogue between an artist and a time traveler about cultural diversity,",
"Design a documentary movie plot set in a post-apocalyptic world,",
"Create a dialogue between a chef and a historian about cultural diversity,",
"Create a dialogue between a teacher and a robot about space exploration,",
"Create a dialogue between an artist and an alien about climate change,",
"Design a superhero movie plot set in an underwater civilization,",
"Create a dialogue between a chef and a robot about space exploration,",
"Create a dialogue between a scientist and a robot about climate change,",
"Compare and contrast genetic engineering and cable television,",
"Design a romantic comedy movie plot set in a post-apocalyptic world,",
"Create a marketing slogan for online learning platform,",
"Design a historical drama movie plot set in a futuristic megacity,",
"Compare and contrast cryptocurrencies and face-to-face communication,",
"Explain artificial intelligence to a 10-year-old,",
"Compare and contrast cryptocurrencies and traditional banking,",
"Create a dialogue between an artist and a time traveler about climate change,",
"Create a dialogue between a scientist and a historian about space exploration,",
"Describe a deserted island in vivid detail,",
"Create a dialogue between a scientist and a detective about climate change,",
"Compare and contrast traditional education and traditional banking,",
"Compare and contrast electric cars and traditional banking,",
"Compare and contrast streaming services and cable television,",
"Design a documentary movie plot set in an alien planet,",
"Compare and contrast artificial intelligence and face-to-face communication,",
"Develop a concept for a 1950s retro themed restaurant,",
"Create a dialogue between an artist and a student about artificial intelligence,",
"Compare and contrast social media and traditional banking,",
"Imagine and describe a day in the life of a climate refugee,",
"Compare and contrast electric cars and cable television,",
"Compare and contrast social media and home-cooked meals,",
"Create a dialogue between an artist and a robot about healthy living,",
"Discuss the significance of artificial intelligence in healthcare,",
"Describe how food preservation works,",
"Discuss the significance of cultural diversity in South Africa,",
"Compare and contrast virtual reality and online learning,",
"Compare and contrast cryptocurrencies and human intelligence,",
"Explain climate change to a 15-year-old,",
"Create a dialogue between an artist and a detective about healthy living,",
"Explain photosynthesis to a 5-year-old,",
"Create a dialogue between a chef and a child about artificial intelligence,",
"Design a crime mystery movie plot set in ancient Egypt,",
"Create a dialogue between a politician and a retiree about climate change,",
"Compose a poem about healthy living,",
"Create a dialogue between an artist and a child about space exploration,",
"Compose a poem about artificial intelligence,",
"Create a dialogue between a teacher and a retiree about artificial intelligence,",
"Design a crime mystery movie plot set in an alien planet,",
"Create a dialogue between an artist and an alien about healthy living,",
"Create a dialogue between a chef and a historian about space exploration,",
"Explain evolution to a 5-year-old,",
"Create a dialogue between a scientist and an alien about cultural diversity,",
"Create a dialogue between a teacher and a time traveler about cultural diversity,",
"Design a musical movie plot set in an alien planet,",
"Write a news article about artificial intelligence,",
"Create a dialogue between a chef and a philosopher about climate change,",
"Troubleshoot the issue of rotator in a bike,",
"Explain blockchain to a 8-year-old,",
"Create a dialogue between an artist and a child about artificial intelligence,",
"Create a dialogue between a teacher and a philosopher about artificial intelligence,",
"Explain quantum computing to a 8-year-old,",
"Compare and contrast genetic engineering and office-based work,",
"Create a dialogue between an artist and a student about healthy living,",
"Compare and contrast traditional education and natural selection,",
"Compare and contrast streaming services and online learning,",
"Create a dialogue between a chef and a child about climate change,",
"Compare and contrast cryptocurrencies and augmented reality,",
"Create a dialogue between a politician and an alien about space exploration,",
"Create a dialogue between an artist and a child about healthy living,",
"Create a dialogue between a teacher and a detective about artificial intelligence,",
"Create a dialogue between a politician and a retiree about artificial intelligence,",
"Compare and contrast fast food and face-to-face communication,",
"Design a animated adventure movie plot set in ancient Egypt,",
"Create a dialogue between an artist and a celebrity about climate change,",
"Explain photosynthesis to a 15-year-old,",
"Create a dialogue between a teacher and an alien about cultural diversity,",
"Create a dialogue between a teacher and a child about climate change,",
"Create a dialogue between a politician and a historian about space exploration,",
"Create a dialogue between an artist and a time traveler about healthy living,",
"Create a dialogue between an artist and an alien about artificial intelligence,",
"Explain cryptocurrency to a 12-year-old,",
"Create a dialogue between a teacher and a philosopher about space exploration,",
"Create a dialogue between an artist and a philosopher about climate change,",
"Compare and contrast remote work and traditional banking,",
"Create a dialogue between a politician and a time traveler about space exploration,",
"Design a science fiction movie plot set in an alien planet,",
"Compare and contrast traditional education and online learning,",
"Design a psychological thriller movie plot set in an alien planet,",
"Create a dialogue between a scientist and a historian about healthy living,",
"Create a dialogue between an artist and a retiree about climate change,",
"Explain blockchain to a 12-year-old,",
"Design a documentary movie plot set in the roaring twenties,",
"Create a dialogue between a teacher and a student about climate change,",
"Compare and contrast electric cars and online learning,",
"Imagine and describe a day in the life of a quantum physicist,",
"Design a romantic comedy movie plot set in a magical realm,",
"Create a dialogue between a chef and a philosopher about healthy living,",
"Compare and contrast cryptocurrencies and cable television,",
"Create a dialogue between a chef and a celebrity about space exploration,",
"Compare and contrast cryptocurrencies and office-based work,",
"Analyze the impact of the assassination of Abhraham Lincoln on the US,",
"Create a dialogue between a politician and a retiree about cultural diversity,",
"Create a dialogue between a teacher and a detective about climate change,",
"Design a science fiction movie plot set in a dystopian society,",
"Create a dialogue between a chef and a robot about cultural diversity,",
"Describe a futuristic space colony in vivid detail,",
"Design a historical drama movie plot set in an alien planet,",
"Compare and contrast virtual reality and home-cooked meals,",
"Design a musical movie plot set in a dystopian society,",
"Design a historical drama movie plot set in ancient Egypt,",
"Evaluate the pros and cons of climate change,",
"Compare and contrast fast food and traditional banking,",
"Evaluate the pros and cons of space exploration,",
"Create a dialogue between a chef and a child about space exploration,",
"Develop a concept for a steampunk themed restaurant,",
"Compare and contrast electric cars and gasoline vehicles,",
"Create a dialogue between a politician and a student about artificial intelligence,",
"Create a dialogue between a chef and a historian about healthy living,",
"Design a superhero movie plot set in a magical realm,",
"Compare and contrast social media and online learning,",
"Create a dialogue between a scientist and a historian about cultural diversity,",
"Compare and contrast fast food and online learning,",
"Create a dialogue between a teacher and a time traveler about artificial intelligence,",
"Explain mindfulness to a 15-year-old,",
"Design a horror movie plot set in a magical realm,",
"Design a romantic comedy movie plot set in a futuristic megacity,",
"Create a dialogue between a chef and a celebrity about cultural diversity,",
"Explain artificial intelligence to a 12-year-old,",
"Design a animated adventure movie plot set in a magical realm,",
"Design a documentary movie plot set in a parallel universe,",
"Explain evolution to a 15-year-old,",
"Create a dialogue between an artist and an alien about cultural diversity,",
"Design a horror movie plot set in a futuristic megacity,",
"Explain artificial intelligence to a 8-year-old,",
"Create a dialogue between a scientist and a student about space exploration,",
"Write a news article about climate change,",
"Design a superhero movie plot set in the roaring twenties,",
"Create a dialogue between a teacher and a student about healthy living,",
"Design a horror movie plot set in a post-apocalyptic world,",
"Create a dialogue between a teacher and a student about space exploration,",
"Create a dialogue between a politician and a time traveler about artificial intelligence,",
"Create a dialogue between a chef and a time traveler about artificial intelligence,",
"Compare and contrast remote work and natural selection,",
"Compare and contrast virtual reality and gasoline vehicles,",
"Create a dialogue between a teacher and a retiree about healthy living,",
"Design a documentary movie plot set in a dystopian society,",
"Compare and contrast fast food and cable television,",
"Write a news article about space exploration,",
"Create a dialogue between a chef and a philosopher about artificial intelligence,",
"Create a dialogue between a scientist and a robot about cultural diversity,",
"Compose a poem about space exploration,",
"Compare and contrast streaming services and augmented reality,",
"Design a crime mystery movie plot set in a post-apocalyptic world,",
"Imagine and describe a day in the life of a time-traveling historian,",
"Describe a haunted house in vivid detail,",
"Create a dialogue between a scientist and a child about healthy living,",
"Create a dialogue between a chef and a philosopher about cultural diversity,",
"Create a dialogue between a chef and a student about climate change,",
"Describe how recycling works,",
"Design a animated adventure movie plot set in a post-apocalyptic world,",
"Create a dialogue between a politician and a detective about artificial intelligence,",
"Design a superhero movie plot set in a futuristic megacity,",
"Create a dialogue between a politician and a robot about climate change,",
"Compare and contrast artificial intelligence and home-cooked meals,",
"Evaluate the pros and cons of healthy living,",
"Create a dialogue between a chef and a retiree about healthy living,",
"Create a dialogue between a politician and a philosopher about cultural diversity,",
"Create a marketing slogan for smartwatch,",
"Compare and contrast traditional education and augmented reality,",
"Create a dialogue between a teacher and a historian about cultural diversity,",
"Create a dialogue between a chef and a celebrity about artificial intelligence,",
"Create a dialogue between an artist and a celebrity about cultural diversity,",
"Create a dialogue between a chef and a retiree about cultural diversity,",
"Compose a poem about cultural diversity,",
"Provide a step-by-step guide on how to build a shed,",
"Describe a high-tech laboratory in vivid detail,",
"Design a romantic comedy movie plot set in a dystopian society,",
"Create a dialogue between a politician and a child about cultural diversity,",
"Design a psychological thriller movie plot set in a post-apocalyptic world,",
"Create a dialogue between a teacher and a detective about cultural diversity,",
"Create a dialogue between a politician and a child about healthy living,",
"Design a crime mystery movie plot set in a magical realm,",
"Create a dialogue between an artist and a historian about healthy living,",
"Write a short story about climate change,",
"Create a dialogue between a politician and a robot about cultural diversity,",
"Compare and contrast virtual reality and augmented reality,",
"Compose a poem about climate change,",
"Design a musical movie plot set in a futuristic megacity,",
"Create a dialogue between a teacher and a child about healthy living,",
"Create a dialogue between a politician and a retiree about space exploration,",
"Evaluate the pros and cons of artificial intelligence,",
"Explain climate change to a 8-year-old,",
"Create a dialogue between a scientist and a child about artificial intelligence,",
"Design a psychological thriller movie plot set in a futuristic megacity,",
"Create a dialogue between a politician and a celebrity about healthy living,",
"Create a dialogue between an artist and a historian about cultural diversity,",
"Create a dialogue between a chef and a time traveler about climate change,",
"Design a superhero movie plot set in ancient Egypt,",
"Compare and contrast cryptocurrencies and online learning,",
"Create a dialogue between a scientist and a philosopher about space exploration,",
"Explain evolution to a 12-year-old,",
"Create a dialogue between a scientist and a robot about artificial intelligence,",
"Describe a bustling city market in vivid detail,",
"Create a dialogue between a teacher and a time traveler about climate change,",
"Compare and contrast social media and cable television,",
"Describe a prehistoric landscape in vivid detail,",
"Compare and contrast genetic engineering and home-cooked meals,",
"Create a dialogue between an artist and a celebrity about healthy living,",
"Create a dialogue between a politician and a philosopher about space exploration,",
"Compare and contrast streaming services and office-based work,",
"Compare and contrast artificial intelligence and traditional banking,",
"Design a romantic comedy movie plot set in an alien planet,",
"Create a dialogue between a scientist and a child about space exploration,",
"Create a dialogue between a teacher and a historian about artificial intelligence,",
"Create a dialogue between a chef and a student about artificial intelligence,",
"Create a dialogue between a politician and a student about healthy living,",
"Create a dialogue between a scientist and a child about cultural diversity,",
"Describe how machine learning works,",
"Create a dialogue between an artist and a philosopher about healthy living,",
"Develop a concept for a post-apocalyptic themed restaurant,",
"Describe an underwater research facility in vivid detail,",
"Create a dialogue between a teacher and a robot about climate change,",
"Design a musical movie plot set in a magical realm,",
"Compare and contrast cryptocurrencies and gasoline vehicles,",
"Create a dialogue between a chef and a detective about space exploration,",
"Propose a solution to world hunger,",
"Design a romantic comedy movie plot set in the Wild West,",
"Compare and contrast streaming services and face-to-face communication,",
"Compare and contrast genetic engineering and traditional banking,",
"Create a dialogue between an artist and a retiree about healthy living,",
"Create a dialogue between an artist and a student about cultural diversity,",
"Create a dialogue between a chef and an alien about healthy living,",
"Develop a concept for a cyberpunk themed restaurant,",
"Design a musical movie plot set in a post-apocalyptic world,",
"Create a dialogue between a chef and a historian about climate change,",
"Create a dialogue between an artist and a philosopher about space exploration,",
"Compare and contrast electric cars and natural selection,",
"Compare and contrast virtual reality and cable television,",
"Create a dialogue between an artist and a time traveler about artificial intelligence,",
"Imagine and describe a day in the life of a deep-sea explorer,",
"Create a dialogue between a scientist and a celebrity about cultural diversity,",
"Compare and contrast social media and human intelligence,",
"Design a science fiction movie plot set in an underwater civilization,",
"Create a dialogue between a teacher and a retiree about climate change,",
"Design a historical drama movie plot set in a parallel universe,",
"Create a dialogue between an artist and a celebrity about artificial intelligence,",
"Design a historical drama movie plot set in the Wild West,",
"Compare and contrast streaming services and gasoline vehicles,",
"Explain climate change to a 5-year-old,",
"Write a short story about cultural diversity,",
"Create a dialogue between a chef and a robot about artificial intelligence,",
"Create a dialogue between a chef and a retiree about artificial intelligence,",
"Create a dialogue between an artist and a robot about artificial intelligence,",
"Design a musical movie plot set in ancient Egypt,",
"Create a dialogue between a teacher and a time traveler about healthy living,",
"Create an innovative approach to tackle racism,",
"Compare and contrast genetic engineering and human intelligence,",
"Create a dialogue between a politician and a historian about climate change,",
"Describe how 3D printing works,",
"Design a horror movie plot set in the roaring twenties,",
"Compare and contrast fast food and human intelligence,",
"Design a musical movie plot set in the Wild West,",
"Create a dialogue between a politician and a detective about cultural diversity,",
"Create a marketing slogan for plant-based meat alternative,",
"Create a dialogue between an artist and a robot about climate change,",
"Compare and contrast genetic engineering and natural selection,",
"Design a horror movie plot set in ancient Egypt,",
"Design a psychological thriller movie plot set in a dystopian society,",
"Design a horror movie plot set in a parallel universe,",
"Create a dialogue between a chef and a student about cultural diversity,",
"Compare and contrast virtual reality and traditional banking,",
"Create a dialogue between a chef and a detective about climate change,",
"Compare and contrast traditional education and gasoline vehicles,",
"Design a science fiction movie plot set in a magical realm,",
"Compare and contrast fast food and gasoline vehicles,",
"Summarize the key points of healthy living,",
"Design a animated adventure movie plot set in an alien planet,",
"Compare and contrast remote work and online learning,",
"Compare and contrast cryptocurrencies and natural selection,",
"Compare and contrast fast food and home-cooked meals,",
"Create a dialogue between an artist and a time traveler about space exploration,",
"Compare and contrast streaming services and home-cooked meals,",
"Summarize the key points of climate change,",
"Describe a medieval castle in vivid detail,",
"Develop a concept for a ancient civilization themed restaurant,"
]

# Process the instructions and generate text
data = []
for instruction in tqdm(instructions, desc="Generating responses", unit="instruction"):
    generated_text = generate_text(instruction)
    data.append({
        "instruction": instruction,
        "response": generated_text,
        "model": model_name
    })

# Create a pandas DataFrame
df = pd.DataFrame(data)

# Convert to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Save the dataset
dataset.save_to_disk("hf_hub_dataset")

print("Dataset has been created and saved to the 'hf_hub_dataset' directory.")
print("You can now upload this dataset to the Hugging Face Hub using the `push_to_hub` method.")
print("Example usage:")
print("from huggingface_hub import HfApi")
print("api = HfApi()")
print("api.create_repo('your-username/your-dataset-name', repo_type='dataset')")
print("dataset.push_to_hub('your-username/your-dataset-name')")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating responses:   0%|          | 0/590 [00:00<?, ?instruction/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Generating responses: 100%|██████████| 590/590 [47:55<00:00,  4.87s/instruction]


Saving the dataset (0/1 shards):   0%|          | 0/590 [00:00<?, ? examples/s]

Dataset has been created and saved to the 'hf_hub_dataset' directory.
You can now upload this dataset to the Hugging Face Hub using the `push_to_hub` method.
Example usage:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo('your-username/your-dataset-name', repo_type='dataset')
dataset.push_to_hub('your-username/your-dataset-name')


Check if the Dataset is Proper by Printing some Examples

In [15]:
from datasets import load_from_disk

# Path to the Hugging Face dataset directory
dataset_path = '/teamspace/studios/this_studio/hf_hub_dataset/'

# Load the dataset from disk
dataset = load_from_disk(dataset_path)

# Inspect the dataset
print(dataset)
print(dataset.column_names)
print(dataset[1])  # Inspect the first row


Dataset({
    features: ['instruction', 'response', 'model'],
    num_rows: 590
})
['instruction', 'response', 'model']
{'instruction': 'Explain evolution to a 10-year-old,', 'response': "Instruction: Explain evolution to a 10-year-old,\nResponse: Evolution is the story of how all the different animals and plants on Earth got to be the way they are today. It's like a big puzzle, and scientists are trying to figure out how all the pieces fit together.\n\nA long time ago, there were only a few kinds of animals and plants. Over time, some of these animals and plants changed, and new kinds were born", 'model': 'argilla/notus-7b-v1'}


Script to generate Instructions for the DataSet

In [ ]:
import random

# Set a random seed for reproducibility
random.seed(42)

# Define categories and templates (unchanged)
categories = {
    "Writing": [
        "Write a short story about {topic}",
        "Compose a poem about {topic}",
        "Create a dialogue between {character1} and {character2} about {topic}",
        "Write a news article about {topic}",
        "Describe {scenario} in vivid detail"
    ],
    # ... (other categories remain the same)
}

# Define placeholder values (unchanged)
placeholders = {
    "topic": ["climate change", "artificial intelligence", "space exploration", "healthy living", "cultural diversity"],
    "character1": ["a scientist", "a politician", "a teacher", "an artist", "a chef"],
    "character2": ["a student", "a retiree", "a child", "a celebrity", "a robot"],
    "scenario": ["a day at the beach", "a trip to the moon", "a surprise party", "a mysterious discovery", "an unexpected encounter"]
    # ... (other placeholders remain the same)
}

def generate_instruction():
    category = random.choice(list(categories.keys()))
    template = random.choice(categories[category])
    
    instruction = template
    for placeholder, options in placeholders.items():
        if f"{{{placeholder}}}" in instruction:
            instruction = instruction.replace(f"{{{placeholder}}}", random.choice(options))
    return instruction

def generate_instructions(num_instructions):
    instructions = set()
    max_attempts = num_instructions * 10  # Limit attempts to avoid infinite loop
    attempts = 0
    while len(instructions) < num_instructions and attempts < max_attempts:
        instruction = generate_instruction()
        if instruction not in instructions:
            instructions.add(instruction)
        attempts += 1
    return list(instructions)

# Generate instructions
num_instructions = 2000
instructions = generate_instructions(num_instructions)
random.shuffle(instructions)

# Save instructions to a file
with open("generated_instructions.txt", "w") as f:
    f.write("\n".join(instructions))
print(f"Generated {len(instructions)} unique instructions and saved them to 'generated_instructions.txt'")


Upload the Dataset

In [2]:
from datasets import load_dataset, DatasetDict

from datasets import load_from_disk

# Path to the Hugging Face dataset directory
dataset_path = '/teamspace/studios/this_studio/hf_hub_dataset/'

# Load the dataset from disk
dataset = load_from_disk(dataset_path)

# Push to Hugging Face Hub
dataset.push_to_hub("AINovice2005/NewDataSet")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AINovice2005/NewDataSet/commit/edb153dba1ed6384f5dd3ae5c33c7164bbfbd46b', commit_message='Upload dataset', commit_description='', oid='edb153dba1ed6384f5dd3ae5c33c7164bbfbd46b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/AINovice2005/NewDataSet', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AINovice2005/NewDataSet'), pr_revision=None, pr_num=None)